# Benchmark comparing MongoDB sync vs. async

In [1]:
import time
import asyncio
import contextlib
from pymongo import MongoClient
from motor.motor_asyncio import AsyncIOMotorClient

@contextlib.contextmanager
def timer(description):
    start_time = time.time()
    yield
    end_time = time.time()
    print('{} took {:5.4f} seconds'.format(description, end_time - start_time))
    
sync_client = MongoClient().aiotest.testcol
aio_client = AsyncIOMotorClient().aiotest.testcol
loop = asyncio.get_event_loop()

In [2]:
N = 1000

In [3]:
_ = sync_client.insert_many({'abc': i} for i in range(N))

## Synchronous Requests

In [4]:
with timer('sequential sync read'):
    for i in range(N):
        sync_client.find_one({'abc': i})

sequential sync read took 0.4751 seconds


## Asynchronous Requests

In [5]:
async def read_all_sequential():
    for i in range(N):
        await aio_client.find_one({'abc': i})

with timer('sequential async read'):
    loop.run_until_complete(read_all_sequential())

sequential async read took 0.6313 seconds


In [6]:
for chunk_size in range(1, 20):
    
    async def read_all_chunks():
        for i in range(N // chunk_size):
            await asyncio.gather(*(
                aio_client.find_one({'abc': (i * chunk_size) + j})
                for j in range(chunk_size)))
    
    with timer('parallel async read with {} parallel requests'.format(chunk_size)):           
        loop.run_until_complete(asyncio.gather(read_all_chunks()))

parallel async read with 1 parallel requests took 0.6647 seconds
parallel async read with 2 parallel requests took 0.4478 seconds
parallel async read with 3 parallel requests took 0.3854 seconds
parallel async read with 4 parallel requests took 0.3700 seconds
parallel async read with 5 parallel requests took 0.3487 seconds
parallel async read with 6 parallel requests took 0.3467 seconds
parallel async read with 7 parallel requests took 0.3423 seconds
parallel async read with 8 parallel requests took 0.3332 seconds
parallel async read with 9 parallel requests took 0.3361 seconds
parallel async read with 10 parallel requests took 0.3398 seconds
parallel async read with 11 parallel requests took 0.3277 seconds
parallel async read with 12 parallel requests took 0.3276 seconds
parallel async read with 13 parallel requests took 0.3303 seconds
parallel async read with 14 parallel requests took 0.3282 seconds
parallel async read with 15 parallel requests took 0.3226 seconds
parallel async read

In [7]:
sync_client.drop()